In [227]:
import random
import heapq
import pygame
import random
from a_star_algo import *
from grid_generation import *
from helper import *
from visualization import *

In [228]:
def calculate_fitness(grid, shortest_paths, path_penalty, intersection_penalty):
    fitness_scores = {}

    # Enumerate paths with index starting at 1
    for i, path in enumerate(shortest_paths, start=1):
        # Path length
        path_length = len(path)

        # Intersection count for the current path
        intersection_count = 0
        for (y, x), direction in path:
            # Check bounds and if not on the border
            # Ensure within grid bounds
            if 0 <= y < len(grid) and 0 <= x < len(grid[0]):
                # Not on border
                if 0 < y < len(grid) - 1 and 0 < x < len(grid[0]) - 1:
                    if grid[y][x] == 3:  # Check if the cell value is 3
                        intersection_count += 1

        # Fitness calculation for the current path
        fitness = path_penalty * path_length + intersection_penalty * intersection_count
        fitness_scores[f"Path {i}"] = fitness

    return fitness_scores

In [229]:
def initialize_population(size, width, height, num_buildings, num_emergency_services):
    population = []
    for i in range(size):
        grid = generate_city_grid_with_only_bordering_intersections(
            width, height, num_buildings, num_emergency_services)
        grid_with_intersections = place_intersections_in_every_column_randomly(
            grid)
        population.append(grid_with_intersections)

        # Print the grid for this individual
        print(f"Grid {i + 1}:")
        for row in grid_with_intersections:
            print(row)
        print("\n")  # Add a newline for better separation

    print('Total population size:', len(population))
    return population

In [230]:
# def select_best_grids(population, fitness_scores, num_selected):
#     # Create a list of tuples where each tuple is (grid, average_fitness)
#     average_fitness_scores = []
#     for grid, fitness_dict in zip(population, fitness_scores):
#         # Extract and calculate the average of the fitness scores
#         average_fitness = sum(fitness_dict.values()) / len(fitness_dict)
#         average_fitness_scores.append((grid, average_fitness))

#     # Sort the population based on the average fitness score (ascending order)
#     sorted_population = sorted(average_fitness_scores, key=lambda x: x[1])

#     selected_population = [grid for grid,
#                            _ in sorted_population[:num_selected]]
#     avg_fitness = [avg_fitness for _,
#                    avg_fitness in sorted_population[:num_selected]]

#     # Print the selected grids and their average fitness
#     print("Selected Best Grids:")
#     for i, (grid, fitness) in enumerate(zip(selected_population, avg_fitness), start=1):
#         print(f"Grid {i} (Average Fitness: {fitness}):")
#         for row in grid:
#             print(row)
#         print("\n")  # Add a newline for separation between grids

#     # Return the grids with the best (lowest) average fitness
#     return selected_population, avg_fitness


def select_best_grids(population, fitness_scores, num_selected):
    # Create a list of tuples where each tuple is (grid, average_fitness)
    average_fitness_scores = []
    for grid, fitness_dict in zip(population, fitness_scores):
        # Extract and calculate the average of the fitness scores
        average_fitness = sum(fitness_dict.values()) / len(fitness_dict)
        average_fitness_scores.append((grid, fitness_dict, average_fitness))

    # Sort the population based on the average fitness score (ascending order)
    sorted_population = sorted(average_fitness_scores, key=lambda x: x[2])

    selected_population = [grid for grid, _,
                           _ in sorted_population[:num_selected]]
    selected_fitness_scores = [fitness_dict for _,
                               fitness_dict, _ in sorted_population[:num_selected]]
    avg_fitness = [avg_fitness for _, _,
                   avg_fitness in sorted_population[:num_selected]]

    # Print the selected grids, their fitness scores, and average fitness
    print("Selected Best Grids:")
    for i, (grid, fitness_dict, fitness) in enumerate(zip(selected_population, selected_fitness_scores, avg_fitness), start=1):
        print(f"Grid {i} (Average Fitness: {fitness}):")

        # Print the fitness dictionary for the grid
        print("Fitness Scores (Path : Value):")
        for path, score in fitness_dict.items():
            print(f"  {path}: {score}")

        # Print the grid itself
        print("Grid Layout:")
        for row in grid:
            print(row)

        print("\n")  # Add a newline for separation between grids

    # Return the grids with the best (lowest) average fitness
    return selected_population, avg_fitness

In [231]:
def crossover(fitness1, fitness2):
    """
    Combines two fitness dictionaries into a child fitness dictionary by selecting paths with lower fitness values.

    Parameters:
        fitness1: The fitness dictionary of the first parent grid (e.g., {'path1': 18, 'path2': 17}).
        fitness2: The fitness dictionary of the second parent grid (e.g., {'path1': 16, 'path2': 19}).

    Returns:
        A child fitness dictionary with the lower fitness value for each path.
    """
    child_fitness = {}

    # Iterate through both fitness dictionaries
    for path in fitness1.keys():
        if fitness1[path] <= fitness2[path]:
            child_fitness[path] = fitness1[path]
        else:
            child_fitness[path] = fitness2[path]

    # Debug print: Print the child fitness after crossover
    print("Child fitness after crossover:", child_fitness)

    return child_fitness

In [232]:
def mutate(grid, mutation_rate):
    # for row in range(len(grid)):
    #     for col in range(len(grid[0])):
    #         if grid[row][col] == 0 and random.random() < mutation_rate:
    #             grid[row][col] = 3  # Add an intersection

    # Debug print: Print the grid after mutation
    print("Grid after mutation:")
    for row in grid:
        print(row)

    return grid

In [233]:
def genetic_algorithm(width, height, num_buildings, num_emergency_services, path_penalty, intersection_penalty, population_size, generations, mutation_rate):
    # Initialize population
    
    population = initialize_population(
        population_size, width, height, num_buildings, num_emergency_services)

    for generation in range(generations):
        # Calculate fitness for each grid
        fitness_scores = []
        for grid in population:
            shortest_paths = find_all_shortest_paths(grid)
            fitness = calculate_fitness(
                grid, shortest_paths, path_penalty, intersection_penalty)
            fitness_scores.append(fitness)

        # Select best grids
        selected_population, average_fitness = select_best_grids(
            population, fitness_scores, 6)

        # Crossover to generate new grids
        new_population = []
        while len(new_population) < population_size:
            # Select two parents (grid + fitness dictionary pair)
            parent1, parent2 = random.sample(
                list(zip(selected_population, fitness_scores)), 2
            )
            grid1, fitness1 = parent1
            grid2, fitness2 = parent2

            # Generate child fitness
            child_fitness = crossover(fitness1, fitness2)

            # Generate child grid (optional: modify grid generation logic here if needed)
            child_grid = grid1  # Placeholder; customize if grid must reflect fitness paths

            new_population.append(child_grid)

        # Mutate new population
        population = [mutate(grid, mutation_rate) for grid in new_population]

        # Print best fitness in the generation
        best_fitness = min(average_fitness)
        print(f"Generation {generation + 1}: Best Fitness = {best_fitness}")

    # Return the best grid
    best_index = average_fitness.index(min(average_fitness))
    return population[best_index]

In [234]:
GRID_WIDTH = 15
GRID_HEIGHT = 15
CELL_SIZE = 30
NUM_BUILDINGS = 7
NUM_EMERGENCY = 4


if __name__ == "__main__":
    optimized_grid = genetic_algorithm(
        width=GRID_WIDTH,
        height=GRID_HEIGHT,
        num_buildings=NUM_BUILDINGS,
        num_emergency_services=NUM_EMERGENCY,
        path_penalty=1,
        intersection_penalty=1,
        population_size=10,
        generations=3,
        mutation_rate=0.1
    )
    
    # images = load_images()
    shortest_paths = find_all_shortest_paths(optimized_grid)
    print('Here is the optimized grid')
    for row in optimized_grid:
        print(row)
    # visualize_city_grid_with_offset_paths(
    #     optimized_grid, images, shortest_paths)
    
    

Generating grid with 7 buildings and 4 emergency services...
Grid generated:
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
[3, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 1, 0, 0, 2, 0, 2, 1, 0, 1, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
Grid generated after placing intersecti